In [ ]:
from evaluation import compare_models
compare_models("no_postgame_prob", "model_4_2_25")

In [ ]:
from evaluation import load_pkl_if_exists
odds_df = load_pkl_if_exists("model_4_2_25")
odds_df[odds_df["unit_pnl"] == -1]

,total,min_ou,max_ou,pred,unit_pnl
id,,,,,
401013357,78.0,68.0,68.0,53.153422,-1.0
401014972,59.0,58.5,58.5,52.465247,-1.0
401022510,77.0,57.5,58.5,45.580452,-1.0
401019470,73.0,70.0,70.0,49.147276,-1.0
401014979,52.0,54.5,54.5,62.135646,-1.0
...,...,...,...,...,...
401729786,49.0,49.5,49.5,54.903791,-1.0
401677095,35.0,46.5,47.5,53.476361,-1.0
401677104,75.0,62.5,62.5,53.389582,-1.0


In [ ]:
print(odds_df["total"].std())
print(odds_df["pred"].std())

18.13132894474144
5.882284714678542


In [ ]:
# Next obvious thing is just get more relevant data. 

In [ ]:
import cfbd
import os

CFBD_API_KEY = os.getenv("CFBD_API_KEY")
configuration = cfbd.Configuration(
    host = "https://apinext.collegefootballdata.com",
    access_token=CFBD_API_KEY
)

In [1]:
import pickle as pkl
import pandas as pd
from data.pull_game_team_data import CFBGameTeamData

pd.set_option('display.max_columns', None)

with open("data/pkl_files/game_team_2013_1.pkl", "rb") as f:
    s = pkl.load(f)

In [4]:
cfbgtd = CFBGameTeamData()
cfbgtd.load_game_team_stats_from_pkl_at_year_week(2016, 16)

ValueError: No objects to concatenate

In [ ]:
cfbgtd._gts_expand_to_df(s[0])

category,teamId,team,completionAttempts,firstDowns,fourthDownEff,fumblesLost,fumblesRecovered,interceptionTDs,interceptionYards,interceptions,...,possessionTime,rushingAttempts,rushingTDs,rushingYards,thirdDownEff,totalPenaltiesYards,totalYards,turnovers,yardsPerPass,yardsPerRushAttempt
0,2393,Middle Tennessee,19-33,21,0-1,0,2,NaN,NaN,2,...,23:04,27,0,91,2-8,6-74,309,2,6.6,3.4
1,2426,Navy,3-7,26,2-2,2,0,0,41,0,...,36:56,67,3,366,8-14,2-18,385,2,2.7,5.5


In [ ]:
gts = s[0]

In [ ]:
teams_df = pd.json_normalize(gts.to_dict(), record_path=["teams"], meta="id")
teams_df = teams_df.explode("stats").reset_index(drop=True)
stats_df = pd.json_normalize(teams_df["stats"], meta="id")
final_df = pd.concat([teams_df.drop(columns="stats"), stats_df], axis=1)
flat_df = final_df.pivot(
    index=["id", "teamId", "team"], columns="category", values="stat"
).reset_index()

In [ ]:
from data.data_prep import DataPrep

In [ ]:
dp = DataPrep()

In [ ]:
dp.load_and_patch_errors()